<a href="https://colab.research.google.com/github/benzar80/ProjetWebMeteo/blob/main/TP_Auto_Encodeur_BASHA_DAVENEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TP Apprendre à communiquer avec les auto-encodeurs

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

# Générer des données
1. Générez un vecteur contenant N = 100000 messages aléatoires s ∈ M = {1, 2, · · · , M}.

In [ ]:
N = 100000
M = 8
messages = np.random.randint(1, M+1, N)
print(messages)

 2. Qu’est ce que l’encodage one-hot ?

 3. Appliquez un encodage one-hot pour chaque message.
 La matrice résultante sera donc de taille N × M.

In [ ]:
messages_df = pd.DataFrame({'message': messages})
one_hot_encoded = pd.get_dummies(messages_df['message'])
print(one_hot_encoded)



4. Convertir la matrice en tenseur PyTorch.

In [ ]:
tensor_one_hot = torch.tensor(one_hot_encoded.values)
print(tensor_one_hot)

tensor([[0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        ...,
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.uint8)


# Créer un modèle d’auto-encodeur sous PyTorch
5. Créez l’encodeur à partir du modèle donné par les auteurs de [6]. On notera que la taille d’entrée est égale à
la longueur des vecteurs one-hot, et que la taille de sortie est égale à la longueur d’un nombre complexe. La dernière
couche de normalisation permet de prendre en compte les contraintes énergétiques des transmissions. Il ne faut
pas confondre le sens de normalisation des auteurs, avec celui des statisticiens où les données sont centrées réduites.


In [ ]:
class NormalisationEnergie(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NormalisationEnergie, self).__init__()

    def forward(self, x):
        return f.normalize(x)

In [ ]:
class NormalisationPuissance(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NormalisationPuissance, self).__init__()

    def forward(self, x):
        return x / (m.sqrt(m.mean(x) * m.mean(x)))

In [ ]:
class GaussianNoiseLayer(nn.Module):
    def __init__(self, variance):
        super(GaussianNoiseLayer, self).__init__()
        self.variance = variance

    def forward(self, x):
        noise = torch.randn_like(x) * torch.sqrt(torch.tensor(self.variance))
        noisy_x = x + noise
        return noisy_x

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, input_size, hidden_sizen, sigma_squared):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(M,M),
            nn.ReLU(),
            nn.Linear(M,2),
            NormalisationEnergie(input_size, hidden_sizen)
            #NormalisationPuissance()
        )
        self.noise = GaussianNoiseLayer(
            sigma_squared
        )
        self.decoder = nn.Sequential(
            nn.Linear(2,M),
            nn.ReLU(),
            nn.Linear(M,M),
            nn.Softmax(dim=M),
        )

    def forward(self, x):
        x_encoded = self.encoder(x)
        x_noisy = self.noise(x_encoded)
        x_decoded = self.decoder(x_noisy)
        return x_decoded


6. Expliquez ce qu’est une fonction d’activation.